# FE Stats Classifier Model Analysis

In [1]:
# [FE_Stats_Data_Analysis]: Need to include path to the singleton pattern impl due to a dependency from the CharacterStatsAnalysis class
# Note: The directory is based of the current working dir of the notebook since the class base directory changes to this one
import sys, os

sys.path.append(os.path.abspath('src/Singleton'))

import torch
import torch.utils.data as torch_data
from src.Models.FEMLPClassifier import FEMLPClassifier
from src.Models.run_forward_pytorch import run_train_forward_pytorch
from src.Models.StatsDataset import StatsDataset

accel_device = "cuda:0"

dtype = torch.float32

batch_size = 8


In [2]:
stats_dataset = StatsDataset(accel_device, dtype)
train_dataset, test_dataset = torch_data.random_split(stats_dataset, [0.8, 0.2])
train_dataset, val_dataset = torch_data.random_split(train_dataset, [0.8, 0.2])
train_dataloader = torch_data.DataLoader(train_dataset, batch_size, True)

val_dataloader = torch_data.DataLoader(val_dataset, batch_size, True)

test_dataloader = torch_data.DataLoader(test_dataset, len(test_dataset))


In [6]:
num_features = stats_dataset.stats_data[:, :-1].shape[1]
num_classes = stats_dataset.stats_data[:, -1].unique().shape[0]
classifier = FEMLPClassifier(num_features * 3, 3, num_features, num_classes, accel_device, dtype)
